In [1]:
from Bio.PDB import PDBParser, NeighborSearch
from scipy.spatial import KDTree
import numpy as np

def find_nearby_atoms_fast(input_pdb, rf_pdb, residues_of_interest, distance_threshold):
    # Parse the input PDB files
    parser = PDBParser(QUIET=True)
    input_structure = parser.get_structure("input", input_pdb)
    rf_structure = parser.get_structure("rf", rf_pdb)

    # Get coordinates of atoms from the specified residues in the input PDB
    atoms_of_interest = []
    for model in input_structure:
        for chain in model:
            for residue in chain:
                if residue.get_id()[1] in residues_of_interest:  # Get residue number
                    atoms_of_interest.extend(residue.get_atoms())

    # Convert the list of atoms to an array of coordinates for faster processing
    interest_coords = np.array([atom.coord for atom in atoms_of_interest])

    # Get all atoms from RFdiff1.pdb and their coordinates
    rf_atoms = list(rf_structure.get_atoms())
    rf_coords = np.array([atom.coord for atom in rf_atoms])

    # Use KDTree for efficient nearest-neighbor search
    kd_tree = KDTree(rf_coords)

    # Find atoms within the distance threshold
    nearby_atoms = []
    for idx, coord in enumerate(interest_coords):
        indices = kd_tree.query_ball_point(coord, distance_threshold)
        for i in indices:
            distance = np.linalg.norm(coord - rf_coords[i])
            if distance <= distance_threshold:
                nearby_atoms.append((atoms_of_interest[idx], rf_atoms[i], distance))

    return nearby_atoms

# Parameters
input_pdb = "input.pdb"
rf_pdb = "RFdiff1.pdb"
residues_of_interest = [10, 20, 30]  # Example residue numbers
distance_threshold = 5.0  # Angstroms

# Find nearby atoms
nearby_atoms = find_nearby_atoms_fast(input_pdb, rf_pdb, residues_of_interest, distance_threshold)

# Output the results
for atom1, atom2, distance in nearby_atoms:
    print(f"Input atom {atom1.get_id()} in residue {atom1.get_parent().get_id()[1]} "
          f"is {distance:.2f} Å from RF atom {atom2.get_id()} in residue {atom2.get_parent().get_id()[1]}")

FileNotFoundError: [Errno 2] No such file or directory: 'input.pdb'